## Module 4 : Fonctions de Vulnérabilité et Calcul d'Impact (30 min)

### 4.1 Définition des fonctions de vulnérabilité

In [ ]:
# Import des classes CLIMADA pour les fonctions d'impact
from climada.entity import ImpactFunc, ImpactFuncSet

def create_flood_impact_functions():
    """
    Crée les fonctions de vulnérabilité pour différents types de bâtiments
    adaptées au contexte ivoirien
    """
    impact_funcs = ImpactFuncSet()
    
    # Fonction 1: Résidentiel
    impf_residential = ImpactFunc()
    impf_residential.haz_type = 'FL'  # Flood
    impf_residential.id = 1
    impf_residential.name = 'Résidentiel Abidjan'
    
    # Courbe hauteur d'eau (mm) vs pourcentage de dommage
    # Adaptée aux constructions ivoiriennes (souvent surélevées)
    intensities_res = [0, 200, 500, 1000, 1500, 2000, 3000]  # mm
    damages_res = [0, 0.05, 0.15, 0.35, 0.60, 0.80, 0.90]   # fraction
    
    impf_residential.set_step_impf(intensities_res, damages_res)
    
    # Fonction 2: Commercial
    impf_commercial = ImpactFunc()
    impf_commercial.haz_type = 'FL'
    impf_commercial.id = 2
    impf_commercial.name = 'Commercial Abidjan'
    
    # Plus sensible (stocks, équipements au sol)
    intensities_com = [0, 100, 300, 800, 1500, 2500]  # mm
    damages_com = [0, 0.10, 0.25, 0.50, 0.75, 0.95]  # fraction
    
    impf_commercial.set_step_impf(intensities_com, damages_com)
    
    # Fonction 3: Industriel
    impf_industrial = ImpactFunc()
    impf_industrial.haz_type = 'FL'
    impf_industrial.id = 3
    impf_industrial.name = 'Industriel Abidjan'
    
    # Très sensible (machines, processus)
    intensities_ind = [0, 50, 200, 500, 1000, 2000]  # mm
    damages_ind = [0, 0.15, 0.35, 0.60, 0.85, 0.95]  # fraction
    
    impf_industrial.set_step_impf(intensities_ind, damages_ind)
    
    # Ajout à l'ensemble
    impact_funcs.append(impf_residential)
    impact_funcs.append(impf_commercial)
    impact_funcs.append(impf_industrial)
    
    return impact_funcs

# Création des fonctions d'impact
impact_functions = create_flood_impact_functions()

print("📈 FONCTIONS DE VULNÉRABILITÉ CRÉÉES:")
for impf in impact_functions:
    print(f"  ID {impf.id}: {impf.name}")
    print(f"    Intensité max: {max(impf.intensity):.0f} mm")
    print(f"    Dommage max: {max(impf.mdd)*100:.0f}%")

# Vérification
impact_functions.check()
print("\n✅ Fonctions de vulnérabilité validées!")

### 4.2 Visualisation des courbes de vulnérabilité

In [ ]:
# Graphique des courbes de vulnérabilité
plt.figure(figsize=(12, 8))

colors = ['blue', 'green', 'red']
markers = ['o', 's', '^']

for i, impf in enumerate(impact_functions):
    plt.plot(impf.intensity, impf.mdd * 100, 
             color=colors[i], marker=markers[i], linewidth=2, markersize=8,
             label=impf.name)

plt.xlabel('Hauteur d\'eau (mm)', fontsize=12)
plt.ylabel('Pourcentage de dommage (%)', fontsize=12)
plt.title('Courbes de Vulnérabilité - Inondations Abidjan\n(Adaptées au contexte ivoirien)', 
          fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend(fontsize=11)
plt.xlim(0, 3000)
plt.ylim(0, 100)

# Ajout de lignes de référence
plt.axhline(50, color='orange', linestyle='--', alpha=0.7, label='50% dommage')
plt.axvline(1000, color='purple', linestyle='--', alpha=0.7, label='1m hauteur')

plt.tight_layout()
plt.show()

# Tableau comparatif pour hauteurs d'eau typiques
test_heights = [100, 300, 500, 1000, 1500, 2000]  # mm

print("\n📊 DOMMAGES PAR TYPE DE BÂTIMENT (% de la valeur):")
print("="*70)
print(f"{'Hauteur (mm)':<12} {'Résidentiel':<12} {'Commercial':<12} {'Industriel':<12}")
print("="*70)

for height in test_heights:
    damages = []
    for impf in impact_functions:
        # Interpolation pour obtenir le dommage à cette hauteur
        damage_pct = np.interp(height, impf.intensity, impf.mdd) * 100
        damages.append(damage_pct)
    
    print(f"{height:<12} {damages[0]:<12.1f} {damages[1]:<12.1f} {damages[2]:<12.1f}")

print("\n💡 INTERPRÉTATION:")
print("- Résidentiel: Plus résistant (constructions souvent surélevées)")
print("- Commercial: Sensibilité moyenne (stocks et équipements)")
print("- Industriel: Très sensible (machines et processus critiques)")

### 4.3 Attribution des fonctions de vulnérabilité aux expositions

In [ ]:
# Mapping secteur -> ID fonction de vulnérabilité
secteur_to_impf = {
    'residentiel': 1,
    'commercial': 2,
    'industriel': 3,
    'services': 2,      # Traité comme commercial
    'public': 1,        # Traité comme résidentiel
    'transport': 3      # Traité comme industriel
}

# Attribution des IDs de fonction d'impact
exposures.gdf['impf_FL'] = exposures.gdf['secteur'].map(secteur_to_impf)

# Vérification de l'attribution
print("🔗 ATTRIBUTION FONCTIONS DE VULNÉRABILITÉ:")
attribution_stats = exposures.gdf.groupby(['secteur', 'impf_FL']).size().reset_index(name='count')
for _, row in attribution_stats.iterrows():
    impf_name = impact_functions.get_func('FL', row['impf_FL'])[0].name
    print(f"  {row['secteur']} → ID {row['impf_FL']} ({impf_name}): {row['count']} actifs")

# Vérification qu'aucun actif n'a d'ID manquant
missing_impf = exposures.gdf['impf_FL'].isna().sum()
if missing_impf > 0:
    print(f"⚠️ ATTENTION: {missing_impf} actifs sans fonction de vulnérabilité!")
else:
    print("✅ Tous les actifs ont une fonction de vulnérabilité assignée")

# Mise à jour de l'objet exposures
exposures.check()
print("\n✅ Exposures mises à jour avec succès!")

### 4.4 Calcul de l'impact économique

In [ ]:
# Calcul de l'impact avec CLIMADA
from climada.engine import Impact

# Création de l'objet Impact
impact = Impact()

# Calcul des dommages
print("⚙️ Calcul de l'impact en cours...")
impact.calc(exposures, impact_functions, hazard_flood)

# Résultats globaux
total_damage = impact.aai_agg  # Average Annual Impact aggregated
event_damage = impact.at_event[0]  # Dommage pour l'événement unique

print("\n💥 RÉSULTATS IMPACT ÉCONOMIQUE:")
print("="*50)
print(f"Dommages totaux: {event_damage/1e9:.2f} milliards FCFA")
print(f"Dommages annuels moyens: {total_damage/1e9:.2f} milliards FCFA")
print(f"Fréquence événement: {hazard_flood.frequency[0]:.6f} /an")
print(f"Période de retour: {1/hazard_flood.frequency[0]:.0f} ans")

# Comparaison avec dommages observés ONPC
observed_damage = 18e9  # 18 milliards FCFA selon ONPC
accuracy = (1 - abs(event_damage - observed_damage) / observed_damage) * 100
bias = ((event_damage - observed_damage) / observed_damage) * 100

print(f"\n🎯 VALIDATION AVEC DONNÉES ONPC:")
print(f"Dommages observés (ONPC): {observed_damage/1e9:.1f} milliards FCFA")
print(f"Dommages modélisés: {event_damage/1e9:.1f} milliards FCFA")
print(f"Précision: {accuracy:.1f}%")
print(f"Biais: {bias:+.1f}% ({'surestimation' if bias > 0 else 'sous-estimation'})")

if accuracy >= 90:
    print("✅ EXCELLENTE précision du modèle!")
elif accuracy >= 80:
    print("✅ BONNE précision du modèle")
else:
    print("⚠️ Précision à améliorer")

## Module 5 : Analyse Détaillée des Résultats (30 min)

### 5.1 Analyse spatiale des dommages

In [ ]:
# Extraction des dommages par actif
exposures.gdf['damage_fcfa'] = impact.imp_mat[0, :] * exposures.gdf['value']
exposures.gdf['damage_percent'] = (exposures.gdf['damage_fcfa'] / exposures.gdf['value']) * 100

# Analyse par commune
commune_damage = exposures.gdf.groupby('commune').agg({
    'damage_fcfa': 'sum',
    'value': 'sum',
    'damage_percent': 'mean'
}).round(2)

commune_damage['damage_milliards'] = commune_damage['damage_fcfa'] / 1e9
commune_damage['exposition_milliards'] = commune_damage['value'] / 1e9
commune_damage['taux_dommage'] = (commune_damage['damage_fcfa'] / commune_damage['value']) * 100

# Tri par dommages décroissants
commune_damage = commune_damage.sort_values('damage_fcfa', ascending=False)

print("🏙️ DOMMAGES PAR COMMUNE:")
print("="*90)
print(f"{'Commune':<15} {'Dommages (Mds)':<15} {'Exposition (Mds)':<16} {'Taux (%)':<10}")
print("="*90)

for commune, row in commune_damage.iterrows():
    print(f"{commune:<15} {row['damage_milliards']:<15.2f} {row['exposition_milliards']:<16.1f} {row['taux_dommage']:<10.1f}")

# Analyse par secteur
secteur_damage = exposures.gdf.groupby('secteur').agg({
    'damage_fcfa': 'sum',
    'value': 'sum'
})

secteur_damage['damage_milliards'] = secteur_damage['damage_fcfa'] / 1e9
secteur_damage['taux_dommage'] = (secteur_damage['damage_fcfa'] / secteur_damage['value']) * 100
secteur_damage = secteur_damage.sort_values('damage_fcfa', ascending=False)

print(f"\n🏢 DOMMAGES PAR SECTEUR:")
print("="*70)
print(f"{'Secteur':<15} {'Dommages (Mds)':<15} {'Taux (%)':<10}")
print("="*70)

for secteur, row in secteur_damage.iterrows():
    print(f"{secteur:<15} {row['damage_milliards']:<15.2f} {row['taux_dommage']:<10.1f}")

# Statistiques générales
print(f"\n📊 STATISTIQUES GÉNÉRALES:")
print(f"Nombre d'actifs endommagés: {(exposures.gdf['damage_fcfa'] > 0).sum():,}")
print(f"Pourcentage d'actifs endommagés: {(exposures.gdf['damage_fcfa'] > 0).mean()*100:.1f}%")
print(f"Dommage moyen par actif touché: {exposures.gdf[exposures.gdf['damage_fcfa'] > 0]['damage_fcfa'].mean()/1e6:.1f} millions FCFA")
print(f"Dommage maximum: {exposures.gdf['damage_fcfa'].max()/1e6:.1f} millions FCFA")

### 5.2 Visualisations des résultats

In [ ]:
# Création de visualisations complètes
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Graphique 1: Dommages par commune
ax1 = axes[0, 0]
commune_damage['damage_milliards'].plot(kind='bar', ax=ax1, color='coral')
ax1.set_title('Dommages par Commune\n(milliards FCFA)', fontweight='bold')
ax1.set_ylabel('Dommages (milliards FCFA)')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(True, alpha=0.3)

# Graphique 2: Taux de dommage par commune
ax2 = axes[0, 1]
commune_damage['taux_dommage'].plot(kind='bar', ax=ax2, color='lightblue')
ax2.set_title('Taux de Dommage par Commune\n(% de la valeur exposée)', fontweight='bold')
ax2.set_ylabel('Taux de dommage (%)')
ax2.tick_params(axis='x', rotation=45)
ax2.grid(True, alpha=0.3)

# Graphique 3: Dommages par secteur
ax3 = axes[1, 0]
secteur_damage['damage_milliards'].plot(kind='pie', ax=ax3, autopct='%1.1f%%')
ax3.set_title('Répartition Dommages par Secteur', fontweight='bold')
ax3.set_ylabel('')

# Graphique 4: Distribution spatiale des dommages
ax4 = axes[1, 1]
scatter = ax4.scatter(exposures.gdf['longitude'], exposures.gdf['latitude'],
                     c=exposures.gdf['damage_fcfa']/1e6, cmap='Reds', 
                     s=30, alpha=0.7)
ax4.set_xlabel('Longitude')
ax4.set_ylabel('Latitude')
ax4.set_title('Distribution Spatiale des Dommages', fontweight='bold')
cbar = plt.colorbar(scatter, ax=ax4)
cbar.set_label('Dommages (millions FCFA)')

plt.tight_layout()
plt.show()

# Sauvegarde des résultats détaillés
commune_damage.to_csv('results/dommages_par_commune.csv')
secteur_damage.to_csv('results/dommages_par_secteur.csv')
exposures.gdf[['commune', 'secteur', 'value', 'damage_fcfa', 'damage_percent']].to_csv('results/dommages_detailles.csv')

print("\n💾 RÉSULTATS SAUVEGARDÉS:")
print("  - results/dommages_par_commune.csv")
print("  - results/dommages_par_secteur.csv")
print("  - results/dommages_detailles.csv")

### 5.3 Rapport de synthèse et recommandations

In [ ]:
# Génération du rapport de synthèse
rapport = f"""
RAPPORT DE SYNTHÈSE - MODÉLISATION INONDATIONS ABIDJAN JUIN 2018
================================================================

1. ÉVÉNEMENT ANALYSÉ
   - Date: 19-20 juin 2018
   - Précipitations maximales: {max_24h:.1f} mm/24h (Port-Bouët)
   - Période de retour estimée: {return_period_event:.0f} ans

2. RÉSULTATS MODÉLISATION
   - Dommages modélisés: {event_damage/1e9:.2f} milliards FCFA
   - Dommages observés (ONPC): {observed_damage/1e9:.1f} milliards FCFA
   - Précision du modèle: {accuracy:.1f}%
   - Biais: {bias:+.1f}%

3. RÉPARTITION DES DOMMAGES
   Communes les plus affectées:
"""

# Ajout des 3 communes les plus affectées
for i, (commune, row) in enumerate(commune_damage.head(3).iterrows()):
    rapport += f"   {i+1}. {commune}: {row['damage_milliards']:.2f} milliards FCFA ({row['taux_dommage']:.1f}%)\n"

rapport += f"""
   Secteurs les plus affectés:
"""

for i, (secteur, row) in enumerate(secteur_damage.head(3).iterrows()):
    rapport += f"   {i+1}. {secteur}: {row['damage_milliards']:.2f} milliards FCFA ({row['taux_dommage']:.1f}%)\n"

rapport += f"""
4. RECOMMANDATIONS
   - Renforcement drainage dans les communes les plus vulnérables
   - Amélioration des normes de construction en zones inondables
   - Développement système d'alerte précoce
   - Révision des plans d'aménagement urbain

5. LIMITES ET AMÉLIORATIONS
   - Intégrer données topographiques haute résolution
   - Affiner les fonctions de vulnérabilité par enquêtes terrain
   - Considérer les mesures de protection existantes
   - Analyser les effets de cascade (transport, services)

Date du rapport: {datetime.now().strftime('%d/%m/%Y %H:%M')}
Générateur: CLIMADA Formation DGE
"""

print(rapport)

# Sauvegarde du rapport
with open('results/rapport_synthese_inondations_abidjan.txt', 'w', encoding='utf-8') as f:
    f.write(rapport)

print("\n💾 Rapport sauvegardé: results/rapport_synthese_inondations_abidjan.txt")

# Résumé des fichiers générés
print("\n📁 FICHIERS GÉNÉRÉS PENDANT CE TP:")
print("  📊 Données:")
print("    - exposition_par_commune.csv")
    - dommages_par_commune.csv")
print("    - dommages_par_secteur.csv")
print("    - dommages_detailles.csv")
print("  🗺️ Cartes:")
print("    - carte_aleas_inondation_abidjan.html")
print("  📄 Rapports:")
print("    - rapport_synthese_inondations_abidjan.txt")

print("\n🎯 TP1 TERMINÉ AVEC SUCCÈS!")
print("Vous maîtrisez maintenant le workflow complet CLIMADA pour les inondations urbaines.")